In [1]:
import numpy as np
from pprint import pprint
import pdir

In [12]:
def gen_ids(f_names):
    words=[]
    labels=[]
    for line in open(f_names,"r",encoding="utf-8"):
        label,sentence=line.split("\t")
        labels.append(int(label))
        for w in sentence.split():
            words.append(w)
    words=set(words)
    w_ids={}
    
    for w in words:
        w_ids[w]=len(w_ids)
    sentences=[]
    for line in open(f_names,"r",encoding="utf-8"):
        label,sentence=line.split("\t")
        sent=np.zeros(len(w_ids))
        for w in sentence.split():
            sent[w_ids[w]]+=1
        sentences.append(sent)
    return w_ids,sentences,labels

In [17]:
w_ids,sentences,labels=gen_ids("../../test/03-train-input.txt")

In [18]:
sentences

[array([ 1.,  1.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  2.]),
 array([ 0.,  1.,  0.,  1.,  0.,  1.,  0.,  1.,  1.,  1.])]

In [68]:
class NN():
    from pprint import pprint

    def __init__(self, input_list, labels, network_dims):
        assert len(input_list) == len(labels)

        self.input_list = input_list
        self.labels = labels
        self.network_neuron_outputs = []
        self.network_neuron_nets = []

        self.network_neuron_derr_net = []
        self.netword_neuron_dnet_weight = []

        self.network_weights = []

        dim_prev = len(input_list[0])
        for i in range(len(network_dims)):
            dim = network_dims[i]
            self.network_weights.append(np.random.rand(dim_prev, dim))
            self.network_neuron_outputs.append(np.zeros(dim))
            self.network_neuron_nets.append(np.zeros(dim))
            self.network_neuron_derr_net.append(np.zeros(dim))
            self.netword_neuron_dnet_weight.append(np.zeros(dim))
            dim_prev=dim
            
        self.network = (self.network_neuron_outputs, self.network_neuron_nets,
                        self.network_neuron_derr_net,
                        self.netword_neuron_dnet_weight)
        
    def print_network(self):
        for w in self.__dict__:
            if w.startswith("network"):
                print(w)
                pprint(self.__dict__[w])
                print()

    def ff_one(self, input_array):
        for idx in range(len(self.network_weights)):
            if idx == 0:
                outputs_prev = input_array
            else:
                outputs_prev = self.network_neuron_outputs[idx - 1]
            self.network_neuron_nets[idx] = np.dot(outputs_prev,
                                                   self.network_weights[idx])
            self.network_neuron_outputs[idx] = np.tanh(
                self.network_neuron_nets[idx])

    def bk_one(self, label):
        assert label.shape[0] == 1
        for idx, grads in enumerate(reversed(self.network_grads)):
            if idx == 0:
                err_grad = label - self.network_neurons[-1]

In [69]:
nn=NN(sentences,labels,(3,1))

In [70]:
nn.print_network()

network_weights
[array([[ 0.63010516,  0.66083425,  0.1121095 ],
       [ 0.79639929,  0.74014279,  0.6474969 ],
       [ 0.50913803,  0.23200858,  0.58972893],
       [ 0.21049672,  0.90638303,  0.45472865],
       [ 0.19972734,  0.17483194,  0.9187023 ],
       [ 0.37450293,  0.19746606,  0.18601212],
       [ 0.40988596,  0.84041012,  0.62264218],
       [ 0.6033717 ,  0.52711194,  0.6221317 ],
       [ 0.1932401 ,  0.00944553,  0.40110175],
       [ 0.42893511,  0.09306512,  0.53902861]]),
 array([[ 0.27824949],
       [ 0.20751352],
       [ 0.57625873]])]

network
([array([ 0.,  0.,  0.]), array([ 0.])],
 [array([ 0.,  0.,  0.]), array([ 0.])],
 [array([ 0.,  0.,  0.]), array([ 0.])],
 [array([ 0.,  0.,  0.]), array([ 0.])])

network_neuron_outputs
[array([ 0.,  0.,  0.]), array([ 0.])]

network_neuron_derr_net
[array([ 0.,  0.,  0.]), array([ 0.])]

network_neuron_nets
[array([ 0.,  0.,  0.]), array([ 0.])]



In [74]:
nn.ff_one(nn.input_list[0])

In [75]:
nn.print_network()

network_weights
[array([[ 0.63010516,  0.66083425,  0.1121095 ],
       [ 0.79639929,  0.74014279,  0.6474969 ],
       [ 0.50913803,  0.23200858,  0.58972893],
       [ 0.21049672,  0.90638303,  0.45472865],
       [ 0.19972734,  0.17483194,  0.9187023 ],
       [ 0.37450293,  0.19746606,  0.18601212],
       [ 0.40988596,  0.84041012,  0.62264218],
       [ 0.6033717 ,  0.52711194,  0.6221317 ],
       [ 0.1932401 ,  0.00944553,  0.40110175],
       [ 0.42893511,  0.09306512,  0.53902861]]),
 array([[ 0.27824949],
       [ 0.20751352],
       [ 0.57625873]])]

network
([array([ 0.99849707,  0.99324749,  0.99967984]), array([ 0.78567068])],
 [array([ 3.59636609,  2.84380345,  4.36983879]), array([ 1.06001782])],
 [array([ 0.,  0.,  0.]), array([ 0.])],
 [array([ 0.,  0.,  0.]), array([ 0.])])

network_neuron_outputs
[array([ 0.99849707,  0.99324749,  0.99967984]), array([ 0.78567068])]

network_neuron_derr_net
[array([ 0.,  0.,  0.]), array([ 0.])]

network_neuron_nets
[array([ 3.5963

In [57]:
np.dot(a, b)

array([ 4.27622422,  3.85662286,  2.56456984])

In [16]:
c.shape

(4, 3)

In [1]:
t="""P. Noyes and others, "Low Level Exposure to the Flame Retardant BDE-209 Reduces Thyroid Hormone Levels and Disrupts Thyroid Signaling in Fathead Minnows", Environmental Science & Technology, Vol. 47, No. 17 (2013), pp. 10012−10021.
"""

In [4]:
t.decode("utf-8").encode("utf-8")

'P. Noyes and others, "Low Level Exposure to the Flame Retardant BDE-209 Reduces Thyroid Hormone Levels and Disrupts Thyroid Signaling in Fathead Minnows", Environmental Science & Technology, Vol. 47, No. 17 (2013), pp. 10012\xe2\x88\x9210021.\n'